In [3]:
import pandas as pd
from io import StringIO

# =============================================
# 1. CARREGAR OS DADOS (SIMULANDO LEITURA DE CSV)
# =============================================

# Dados fornecidos no formato string para simular um arquivo CSV
dados_csv = """
ID;Duration;Date;Pulse;Maxpulse;Calories
0;60;'2020/12/01';110;130;4091
1;60;'2020/12/02';117;145;4790
2;60;'2020/12/03';103;135;3400
3;45;'2020/12/04';109;175;2824
4;45;'2020/12/05';117;148;4060
5;60;'2020/12/06';102;127;3000
6;60;'2020/12/07';110;136;3740
7;450;'2020/12/08';104;134;2533
8;30;'2020/12/09';109;133;1951
9;60;'2020/12/10';98;124;2690
10;60;'2020/12/11';103;147;3293
11;60;'2020/12/12';100;120;2507
12;60;'2020/12/12';100;120;2507
13;60;'2020/12/13';106;128;3453
14;60;'2020/12/14';104;132;3793
15;60;'2020/12/15';98;123;2750
16;60;'2020/12/16';98;120;2152
17;60;'2020/12/17';100;120;3000
18;45;'2020/12/18';90;112;NaN
19;60;'2020/12/19';103;123;3230
20;45;'2020/12/20';97;125;2430
21;60;'2020/12/21';108;131;3642
22;45;NaN;100;119;2820
23;60;'2020/12/23';130;101;3000
24;45;'2020/12/24';105;132;2460
25;60;'2020/12/25';102;126;3345
26;60;20201226;100;120;2500
27;60;'2020/12/27';92;118;2410
28;60;'2020/12/28';103;132;NaN
29;60;'2020/12/29';100;132;2800
30;60;'2020/12/30';102;129;3803
31;60;'2020/12/31';92;115;2430
"""

# Carregar os dados em um DataFrame
df = pd.read_csv(StringIO(dados_csv), sep=';', quotechar="'")

# =============================================
# 2. EXPLORAÇÃO INICIAL DOS DADOS
# =============================================

print("\n=== INFORMACOES GERAIS ===")
print(df.info())

print("\n=== PRIMEIRAS 5 LINHAS ===")
print(df.head())

print("\n=== ULTIMAS 5 LINHAS ===")
print(df.tail())

# =============================================
# 3. CRIAR CÓPIA PARA MANIPULAÇÃO
# =============================================

df_clean = df.copy()

# =============================================
# 4. TRATAMENTO DE VALORES AUSENTES (NaN)
# =============================================

# Substituir NaN na coluna 'Calories' por 0
df_clean['Calories'] = df_clean['Calories'].fillna(0)

print("\n=== APOS SUBSTITUIR NaN EM CALORIES ===")
print(df_clean[df_clean['Calories'] == 0])  # Mostrar linhas afetadas

# Substituir NaN na coluna 'Date' por '1900/01/01' (temporário)
df_clean['Date'] = df_clean['Date'].fillna("'1900/01/01'")

print("\n=== APOS SUBSTITUIR NaN EM DATE ===")
print(df_clean[df_clean['Date'] == "'1900/01/01'"])  # Mostrar linhas afetadas

# =============================================
# 5. CONVERSÃO PARA DATETIME (COM TRATAMENTO DE ERROS)
# =============================================

# Tentativa inicial (vai falhar)
try:
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format="'%Y/%m/%d'")
except ValueError as e:
    print(f"\nERRO ENCONTRADO: {e}")

# Corrigir substituindo '1900/01/01' por pd.NA
df_clean['Date'] = df_clean['Date'].replace("'1900/01/01'", pd.NA)

# Tentar novamente (ainda vai falhar por causa de "20201226")
try:
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format="'%Y/%m/%d'")
except ValueError as e:
    print(f"\nNOVO ERRO ENCONTRADO: {e}")

# Remover aspas simples para facilitar o processamento
df_clean['Date'] = df_clean['Date'].str.replace("'", "")

# Converter valor específico "20201226" para datetime
mask = df_clean['Date'] == '20201226'
df_clean.loc[mask, 'Date'] = pd.to_datetime(df_clean.loc[mask, 'Date'], format='%Y%m%d').astype(str)

# Converter toda a coluna para datetime (agora deve funcionar)
df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')

print("\n=== APOS CONVERSAO PARA DATETIME ===")
print(df_clean.head())

# =============================================
# 6. REMOÇÃO DE REGISTROS COM VALORES NULOS
# =============================================

# Remover linhas onde 'Date' é nulo (linha 22 originalmente)
df_clean = df_clean.dropna(subset=['Date'])

print("\n=== APOS REMOCAO DE LINHAS COM DATE NULO ===")
print(df_clean.info())

# =============================================
# 7. VERIFICAÇÃO FINAL
# =============================================

print("\n=== DATAFRAME FINAL ===")
print(df_clean)

print("\n=== INFORMACOES FINAIS ===")
print(df_clean.info())

print("\n=== PRIMEIRAS 5 LINHAS DO RESULTADO FINAL ===")
print(df_clean.head())

print("\n=== ULTIMAS 5 LINHAS DO RESULTADO FINAL ===")
print(df_clean.tail())

# =============================================
# 8. SALVAR RESULTADO (OPCIONAL)
# =============================================
# df_clean.to_csv('dados_tratados.csv', index=False)


=== INFORMACOES GERAIS ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        32 non-null     int64  
 1   Duration  32 non-null     int64  
 2   Date      31 non-null     object 
 3   Pulse     32 non-null     int64  
 4   Maxpulse  32 non-null     int64  
 5   Calories  30 non-null     float64
dtypes: float64(1), int64(4), object(1)
memory usage: 1.6+ KB
None

=== PRIMEIRAS 5 LINHAS ===
   ID  Duration        Date  Pulse  Maxpulse  Calories
0   0        60  2020/12/01    110       130    4091.0
1   1        60  2020/12/02    117       145    4790.0
2   2        60  2020/12/03    103       135    3400.0
3   3        45  2020/12/04    109       175    2824.0
4   4        45  2020/12/05    117       148    4060.0

=== ULTIMAS 5 LINHAS ===
    ID  Duration        Date  Pulse  Maxpulse  Calories
27  27        60  2020/12/27     92       118    24